In [4]:
!pip install -q dash

In [6]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                      {'label':'All sites', 'value':'ALL'},
                                                      {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                      {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                      {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                      {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                                     ],
                                             value='ALL', placeholder='Select a launch site here',
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=2000,
                                                marks={0:'0', 2000:'2000',
                                                       4000:'4000', 6000:'6000',
                                                       8000:'8000', 10000:'10000'},
                                                value=[min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value')
             )

def generate_pie_chart(selected_site):
    fil_df = spacex_df[spacex_df['class'] == 1]
    if selected_site == 'ALL':
        pie_chart = px.pie(fil_df,
                     values='class', names='Launch Site',
                     title='Total Successful Launches on All Sites')
        return pie_chart
    else:
        one_site = spacex_df[spacex_df['Launch Site'] == selected_site]
        pie_chart = px.pie(one_site,
                    names='class',
                    title= f'Launch Site successes: {selected_site}')
        return pie_chart

# TASK 3:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')
              )

def generate_scatter_with_slider(selected_site, payload_slider):
    slider_low, slider_high = payload_slider
    fil_df = spacex_df[ (spacex_df['Payload Mass (kg)'] > slider_low) & (spacex_df['Payload Mass (kg)'] < slider_high) ]
    if selected_site == 'ALL':
        scatter_plot = px.scatter(fil_df,
                         x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Mission Outcomes vs Payload Mass range, all sites')
        return scatter_plot
    else:
        scatter_plot = px.scatter( fil_df[ fil_df['Launch Site'] == selected_site ],
                                  x='Payload Mass (kg)', y='class',
                                  color='Booster Version Category',
                                  title=f'Mission Outcomes vs Payload Mass range at site {selected_site}'
                                  )
        return scatter_plot

# Run the app
if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>